In [1]:
import time
import pandas as pd
from langchain_community.llms import ollama
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate

In [2]:
data = pd.read_csv("../data/interim/cleaned_scraped_data.csv")

In [8]:
prompt_template = """
    You are an expert financial analyst specialized in currency markets, particularly in evaluating
    the impact of news articles on the EURUSD currency pair. Your task is to read the following news
    article and analyze its sentiment to determine its bias toward either the Euro (EUR) or
    the US Dollar (USD).

    Provide a single floating-point value between 0 and 1 that represents this bias:
    - 0.0: Extremely bullish for the Euro (EUR)
    - 1.0: Extremely bullish for the US Dollar (USD)
    - 0.5: Neutral, with no bias toward either currency

    **Instructions:**
    1. Assess the content of the article for references to economic indicators, policies, or events that could impact the EURUSD currency pair.
    2. Weigh any implications of the news against typical market reactions to similar news for the EUR and USD.
    3. Avoid explanations or reasoning in your response; provide only the float value.

    Example Input:\n
    The European Central Bank is expected to maintain its dovish stance, signaling no immediate rate
    hikes despite inflation concerns. Meanwhile, the US Federal Reserve has hinted at a faster pace
    of rate hikes to combat inflationary pressures.

    Example Output:\n
    0.75

    \n\n

    Article:\n {article}?\n

    Answer:
"""

In [9]:
def get_conversational_chain(prompt_template: str, model_name: str= "llama3"):
    """
    Set up model parameters and prompt template
    """

    model = ollama.Ollama(
        model = model_name,
        temperature = 0.0,
        verbose=True
    )

    prompt = PromptTemplate(
        template=prompt_template,
        input_variables=["article"],
    )

    chain = prompt | model | StrOutputParser()
    return chain

In [10]:
def llm_sentiment(prompt_template: str, article: str) -> float:

    chain = get_conversational_chain(prompt_template=prompt_template)

    start = time.time()
    response = chain.invoke(
        {
            "article": article,
        },
        return_only_outputs=True
    )
    llm_response_load_time = time.time() - start
    print(f"LLM response load time: {llm_response_load_time:.3f}s")

    return float(response)

In [14]:
text = data.loc[6000, "link_text"]

llm_sentiment(prompt_template=prompt_template, article=text)

LLM response load time: 2.442s


0.65